Import methods from API

In [1]:
from BioTranslator import setup_config, train_text_encoder, get_ontology_embeddings, train_biotranslator, test_biotranslator

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12463\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Train text encoder

In [2]:
model_path = './Codebase/TextEncoder/model/text_encoder.pth'
graphine_repo = './Codebase/TextEncoder/data/Graphine/dataset/'
train_text_encoder(graphine_repo, model_path)

Using cuda device
Batch Size: 16, Max Length: 256


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|▏         | 3/227 [00:00<00:03, 59.66it/s]


UnicodeDecodeError: 'gbk' codec can't decode byte 0xa4 in position 33: illegal multibyte sequence

Build configs

In [3]:
import os
def create_repo(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Warning: We created the repo: {}'.format(path))
seq_repo = f'./Codebase/Protein'
create_repo(seq_repo)
seq_config = {
    'task': 'few_shot',
    'max_length': 2000,
    'data_repo': f'{seq_repo}/data/',
    'dataset': 'GOA_Human',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 1500,
    'features': 'seqs, description, network',
    'lr': 0.0003,
    'epoch': 30,
    'batch_size': 32,
    'gpu_ids': '0',
}
vec_repo = f'./Codebase/SingleCell'
vec_config = {
    'task': 'cross-dataset',
    'eval_dataset': 'muris_facs',
    'vec_ontology_repo': f'{vec_repo}/data/Ontology_data/',
    'data_repo': f'{seq_repo}/data/sc_data/',
    'dataset': 'muris_droplet',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 30,
    'lr': 0.0001,
    'epoch': 15,
    'batch_size': 128,
    'gpu_ids': '0',
}

graph_repo = f'./Codebase/Pathway'
graph_config = {
    'max_length': 2000,
    'eval_dataset': 'KEGG',
    'graph_excludes': ['Reactome', 'KEGG', 'PharmGKB'],
    'data_repo': f'{seq_repo}/data/',
    'dataset': 'GOA_Human',
    'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth',
    'rst_dir': f'{seq_repo}/results/',
    'emb_dir': f'{seq_repo}/embeddings/',
    'ws_dir': f'{seq_repo}/working_space/',
    'hidden_dim': 1500,
    'features': 'seqs, description, network',
    'lr': 0.0003,
    'epoch': 30,
    'batch_size': 32,
    'gpu_ids': '0',
}

seq_config = setup_config(seq_config, 'seq')
vec_config = setup_config(vec_config, 'vec')
graph_config = setup_config(graph_config, 'graph')

cfgs = [seq_config, vec_config, graph_config]

{'task': 'few_shot', 'max_length': 2000, 'features': 'seqs, description, network', 'data_repo': './Codebase/Protein/data/', 'dataset': 'GOA_Human', 'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth', 'rst_dir': './Codebase/Protein/results/', 'emb_dir': './Codebase/Protein/embeddings/', 'ws_dir': './Codebase/Protein/working_space/', 'hidden_dim': 1500, 'lr': 0.0003, 'epoch': 30, 'batch_size': 32, 'gpu_ids': '0'}
{'task': 'cross-dataset', 'eval_dataset': 'muris_facs', 'vec_ontology_repo': './Codebase/SingleCell/data/Ontology_data/', 'data_repo': './Codebase/Protein/data/sc_data/', 'dataset': 'muris_droplet', 'encoder_path': './Codebase/TextEncoder/Encoder/text_encoder.pth', 'rst_dir': './Codebase/Protein/results/', 'emb_dir': './Codebase/Protein/embeddings/', 'ws_dir': './Codebase/Protein/working_space/', 'hidden_dim': 30, 'lr': 0.0001, 'epoch': 15, 'batch_size': 128, 'gpu_ids': '0'}
{'max_length': 2000, 'eval_dataset': 'KEGG', 'graph_excludes': ['Reactome', 'KEGG', 'Pharm

Get Ontology Embeddings

In [4]:
data_dir = seq_config.data_repo + seq_config.dataset + '/'
text_embs = []
for cfg in cfgs:
    get_ontology_embeddings(model_path, data_dir, cfg)

100%|██████████| 47340/47340 [00:00<00:00, 400111.94it/s]
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/47

ImportError: cannot import name 'OpOverloadPacket' from 'torch._ops' (C:\Users\12463\miniconda3\envs\btp\lib\site-packages\torch\_ops.py)

Train BioTranslators

In [5]:
encoders = train_biotranslator(cfgs)

terms number:21656


100%|██████████| 3/3 [00:00<00:00,  3.02it/s]


Rank of your embeddings is 768
Data Loading Finished!


100%|██████████| 2330/2330 [00:00<00:00, 358185.32it/s]
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/2330

ImportError: cannot import name 'OpOverloadPacket' from 'torch._ops' (C:\Users\12463\miniconda3\envs\btp\lib\site-packages\torch\_ops.py)

Test BioTranslators

In [6]:
import pandas as pd
from BioTranslator import BioLoader

tasks = dict(
    seq = ['prot_func_pred'],
    vec = ['cell_type_cls'],
    graph = ['node_cls', 'edge_pred'])
vec_files = BioLoader(vec_config)
anno_data = dict(
    seq = [pd.read_pickle(f'{seq_config.data_repo}{seq_config.dataset}/validation_data_fold_0.pkl')],
    vec = [vec_files.test_data],
    graph = [pd.read_pickle(f'{graph_config.data_repo}{graph_config.eval_dataset}/pathway_dataset.pkl'),
             pd.read_pickle(f'{graph_config.data_repo}{graph_config.eval_dataset}/pathway_dataset.pkl')],
)
for tp_idx, tp in enumerate(list(tasks.keys())):
    for task_idx in range(len(tasks[tp])):
        cfg = cfgs[tp_idx]
        encoder = encoders[tp_idx]
        annos = test_biotranslator(data_dir, anno_data[tp][task_idx], cfg, encoder, tasks[tp][task_idx])
print(annos)

100%|██████████| 2330/2330 [00:00<?, ?it/s]
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/2330 [00:00<?, ?

ImportError: cannot import name 'OpOverloadPacket' from 'torch._ops' (C:\Users\12463\miniconda3\envs\btp\lib\site-packages\torch\_ops.py)